# Week 2 Day 2
We're going to build a simple Agent system for generating cold outreach emails:
    1. Agent workflow'
    2. Use of tools to call functions
    3. Agent collaboration via Tools and Handoffs

In [9]:
# Necessary Imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [3]:
# Load environment
load_dotenv(override=True)

True

# Step 1: Agent WorkFlow

In [4]:
# Create Agent Instructions
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
# Create Agents for each instruction

model_name="gpt-4o-mini"

# Professional Sales Agent
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model_name
)

# Engaging Sales Agent
sales_agent2 = Agent(
    "Engaging Sales Agent",
    instructions=instructions2,
    model=model_name
)

# Busy Sales Agent
sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model_name
)


In [ ]:
# Will use streaming API (.run_streamed())
# Run the instructions but call Runner.run_streamed(agent, instructions) in order to stream back results
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email.")

# Runner.run_streamed(agent, instructions) returns a coroutine object so we have to iterate through it 
# and call async in the for loop
async for event in result.stream_events():
    # Need to check that actual text was returned that we can print
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with AI-Powered Solutions

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, where we specialize in providing an innovative SaaS solution specifically designed to streamline SOC 2 compliance efforts. 

Navigating the complexities of compliance can be daunting, and that's where our AI-powered platform comes in. We automate many of the tedious tasks involved in preparing for audits, enabling your team to focus on what truly matters—growing your business.

Here are just a few ways ComplAI can transform your compliance process:

- **Efficiency:** Our AI-driven tool simplifies documentation and evidence collection, saving valuable time during your audit prep.
- **Accuracy:** Reduce the risk of errors with real-time compliance checks and detailed insights.
- **Scalability:** Whether you're a growing startup or an established enterprise, our solution adapts to your organization's needs.

I w

In [14]:
# Going to use Asyncio for parallel agent calliing

message = "Send a cold sales email."

# Use trace to run agents asynchronously
with trace("Parallel Cold Emails"):
    # Will return a collection of results
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# Store outputs
outputs = [result.final_output for result in results]

# Iterate through outputs and print
for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, an innovative SaaS solution that simplifies the process of achieving and maintaining SOC2 compliance.

In today’s regulatory landscape, ensuring compliance can be both time-consuming and complex. ComplAI leverages advanced AI technology to automate key processes, helping organizations like yours streamline compliance efforts, reduce audit preparation time, and minimize risks associated with non-compliance.

Key benefits of our platform include:

- **Automated Documentation**: Easily generate and manage the required documentation for audits.
- **Real-Time Monitoring**: Continuously track and assess compliance readiness with proactive alerts.
- **Expert Support**: Access to seasoned compliance professionals who understand your unique needs.

We would love the opportunity to discuss how ComplAI 

In [15]:
# Create an agent that picks the best cold email
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. Imagine you are a customer and pick the one you are most likely to respond to. Do not give an explanation; reply with the selected email only.",
    model=model_name
)

In [16]:
# Define the message for Runner.run()
message = "Write a cold sales email"

# Run the agents asynchronously
with trace("Selection from sales people"):
    # Returns a list of coroutines
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# Gather the results into one list 
outputs = [result.final_output for result in results]

# Join the email strings from the 'outputs' list
emails = "Cold Sales Emails:\n\n".join(outputs)

# Run the sales picker to pick the best email
best_email = await Runner.run(sales_picker, emails)

# print out the best email
print(f"Best sales email:\n{best_email.final_output}")

Best sales email:
Subject: Stop Stressing Over SOC2 – We’ve Got the AI Magic Wand! 🪄

Hey [First Name],

Ever feel like SOC2 compliance is the ultimate game of hide and seek, except the only thing that’s hiding is your sanity? 😅

At ComplAI, we believe compliance shouldn’t feel like you’re navigating a maze blindfolded. Our AI-powered tool is here to transform your auditing nightmares into seamless dreams. Think of us as your trusty sidekick—like Robin to your Batman, but with less spandex and more streamlined processes. 🦸‍♂️

Imagine:
- Instant document retrieval
- Automated risk assessments
- A stress-free audit process where your only job is to sip coffee and look impressive!

Let’s chat about how we can turn your compliance chaos into clarity (and maybe some chuckles along the way).

How does next week sound? I promise to bring the laughs, you bring the questions!

Cheers,  
[Your Name]  
[Your Job Title]  
ComplAI  
[Your Phone Number]  
[Your LinkedIn Profile]  

P.S. I hear “Hey

# Step 2 and 3: Tools and Agent interactions
- Will now add tools to the mix

- Wrap function with the @function_tool decorator

In [20]:
# Define a function as a tool to send an email using SendGrid

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    # Set the from email
    from_email = Email("twostrokes210business@gmail.com")
    # Set the to email
    to_email = Email("twostrokes210business@gmail.com")
    # Set the email body content
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales Email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return{"status": "success"}


### This has automatically been converted into a tool, with the JSON boilerplate created

In [21]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015B8AB874C0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool